In [ ]:
# this code is generated by the Domino Code Assist toolbar button
import domino_code_assist as dca
dca.init()

Cannot show ipywidgets in text

# Setup

In particular this notebook requires an environment with the following custom dockerfile instructions:
```
RUN pip install mlflow==2.11.3 transformers datasets ipywidgets torch torchvision --upgrade

```

In [1]:
%matplotlib inline

In [2]:
import os

import numpy as np
import pandas as pd
from datasets import load_dataset, load_metric
from huggingface_hub import notebook_login
from matplotlib import pyplot as plt
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
)

import mlflow

## Huggingface login
NOTE: you need create a huggingface account if you don't already have one and use it to generate a token in the next cell

In [3]:
notebook_login()

In [4]:
!git config --global credential.helper store

In [8]:
train_dataset, test_dataset = load_dataset("imdb", split=["train", "test"])

In [6]:
model = AutoModelForSequenceClassification.from_pretrained("mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis")


In [13]:
from mlflow.store.artifact.runs_artifact_repo import RunsArtifactRepository
from mlflow import MlflowClient

client = MlflowClient()

model_name = "financial-news-sentiment-analysis"
registered_model = client.create_registered_model(model_name)
with mlflow.start_run() as run:
    model_info = mlflow.pytorch.log_model(model, "model")
    
    runs_uri = model_info.model_uri
    
    # Create a new model version of the RandomForestRegression model from this run
    desc = "Pretrained Sentiment Analysis model from mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis"
    model_src = RunsArtifactRepository.get_underlying_uri(runs_uri)
    mv = client.create_model_version(model_name, model_src, run.info.run_id, description=desc)
    print("Name: {}".format(mv.name))
    print("Version: {}".format(mv.version))
    print("Description: {}".format(mv.description))
    print("Status: {}".format(mv.status))
    print("Stage: {}".format(mv.current_stage))

2024/04/05 15:37:55 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: financial-news-sentiment-analysis, version 1


Name: financial-news-sentiment-analysis
Version: 1
Description: Pretrained Sentiment Analysis model from mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis
Status: READY
Stage: None


In [14]:
print(mv)

<ModelVersion: aliases=[], creation_timestamp=1712331475651, current_stage='None', description=('Pretrained Sentiment Analysis model from '
 'mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis'), last_updated_timestamp=1712331475651, name='financial-news-sentiment-analysis', run_id='689e921d215448fc9a25d9ea4258db6e', run_link='', source='mlflow-artifacts:/mlflow/689e921d215448fc9a25d9ea4258db6e/artifacts/model', status='READY', status_message='', tags={'mlflow.domino.dataset_info': '660f0d6e69bfe07330b5551d-660f0d6e69bfe07330b5551c',
 'mlflow.domino.environment_id': '66100749be32346c50baeede',
 'mlflow.domino.environment_revision_id': '661012b6be32346c50baef03',
 'mlflow.domino.hardware_tier': 'small-k8s',
 'mlflow.domino.project_id': '660f0d6c69bfe07330b55517',
 'mlflow.domino.project_name': 'sagemaker-demo',
 'mlflow.domino.run_id': '66101582be32346c50baef07',
 'mlflow.domino.run_number': '3',
 'mlflow.domino.user': 'integration-test',
 'mlflow.domino.user_id': '660b09

In [25]:
from transformers import RobertaTokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
text = "I am very excited today."
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)

print(output)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

SequenceClassifierOutput(loss=None, logits=tensor([[-2.9308, -0.6459,  4.3026]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)


In [33]:
from transformers import pipeline
classification = pipeline('text-classification', model=model, tokenizer=tokenizer)

model_name = "financial-news-sentiment-analysis-classification"
#registered_model = client.create_registered_model(model_name)
with mlflow.start_run() as run:
    model_info = mlflow.transformers.log_model(classification, "model")
    
    runs_uri = model_info.model_uri
    
    # Create a new model version of the RandomForestRegression model from this run
    desc = "Pretrained Sentiment Analysis model from mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis -- text classifier"
    model_src = RunsArtifactRepository.get_underlying_uri(runs_uri)
    mv = client.create_model_version(model_name, model_src, run.info.run_id, description=desc)
    print("Name: {}".format(mv.name))
    print("Version: {}".format(mv.version))
    print("Description: {}".format(mv.description))
    print("Status: {}".format(mv.status))
    print("Stage: {}".format(mv.current_stage))


/tmp/ipykernel_933/2496514756.py:7: FutureWarning: The 'transformers' MLflow Models integration is known to be compatible with the following package version ranges: ``4.25.1`` -  ``4.37.2``. MLflow Models integrations with transformers may not succeed when used with package versions outside of this range.
  model_info = mlflow.transformers.log_model(classification, "model")
/opt/conda/lib/python3.9/site-packages/mlflow/models/model.py:619: FutureWarning: The 'transformers' MLflow Models integration is known to be compatible with the following package version ranges: ``4.25.1`` -  ``4.37.2``. MLflow Models integrations with transformers may not succeed when used with package versions outside of this range.
  flavor.save_model(path=local_path, mlflow_model=mlflow_model, **kwargs)
/opt/conda/lib/python3.9/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This 

Name: financial-news-sentiment-analysis-classification
Version: 1
Description: Pretrained Sentiment Analysis model from mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis -- text classifier
Status: READY
Stage: None


In [34]:
print(mv)

<ModelVersion: aliases=[], creation_timestamp=1712333457686, current_stage='None', description=('Pretrained Sentiment Analysis model from '
 'mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis -- text '
 'classifier'), last_updated_timestamp=1712333457686, name='financial-news-sentiment-analysis-classification', run_id='16e9d6dc49d04f01ac6c55da9620c277', run_link='', source='mlflow-artifacts:/mlflow/16e9d6dc49d04f01ac6c55da9620c277/artifacts/model', status='READY', status_message='', tags={'mlflow.domino.dataset_info': '660f0d6e69bfe07330b5551d-660f0d6e69bfe07330b5551c',
 'mlflow.domino.environment_id': '66100749be32346c50baeede',
 'mlflow.domino.environment_revision_id': '661012b6be32346c50baef03',
 'mlflow.domino.hardware_tier': 'small-k8s',
 'mlflow.domino.project_id': '660f0d6c69bfe07330b55517',
 'mlflow.domino.project_name': 'sagemaker-demo',
 'mlflow.domino.run_id': '66101582be32346c50baef07',
 'mlflow.domino.run_number': '3',
 'mlflow.domino.user': 'integration-te

In [35]:
classification(text)

[{'label': 'positive', 'score': 0.9922448992729187}]